In [1]:
ours2orig = [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27,
28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 254, 41, 42, 43, 44, 45,
46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64,
65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 85,
86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103,
104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118,
119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 136, 138, 132,
133, 134, 135, 220, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152,
153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167,
168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 185, 186,
187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201,
202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216,
217, 218, 219, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232,
233, 137, 234, 235, 236, 237, 238, 239, 240, 241, 250, 251, 255, 256, 257,
258, 259, 260, 261, 262, 263, 242, 243, 244, 245, 0, 1, 2, 3, 4, 5, 6, 7, 8,
9, 10, 11, 83, 84, 131, 139, 140, 141, 181, 182, 183, 184, 246, 247, 248,
249, 252, 253]

def vec2mat(v):
    a,b = np.triu_indices(264,1)
    m = np.zeros((264,264))
    m[a,b] = v
    return m+m.T

def remap(fc, roimap=ours2orig):
    fc = fc[roimap,:]
    fc = fc[:,roimap]
    return fc

print('Complete')

Complete


In [2]:
import pickle
import numpy as np

pncdir = '/home/anton/Documents/Tulane/Research/data-sav/anton/cohorts/PNC/'
lowrankdir = '/home/anton/Documents/Tulane/Research/Work/ContrastiveLearning/PNC/Top10/'
pncdemo = pickle.load(open(f'{pncdir}/demographics.pkl', 'rb'))

rest = []
nback = []
emoid = []
restmat = []
nbackmat = []
emoidmat = []
race = []
sex = []
age = []

a,b = np.triu_indices(264,1)

for sub in pncdemo['age_at_cnb']:
    try:
        ra = pncdemo['Race'][sub]
        ag = pncdemo['age_at_cnb'][sub]
        se = pncdemo['Sex'][sub]
        if ra not in ['AA', 'EA']:
            continue
        ra = ra == 'AA'
        se = se == 'M'
        r = np.load(f'{lowrankdir}/{sub}_task-resttop10_fc.npy')
        n = np.load(f'{lowrankdir}/{sub}_task-nbacktop10_fc.npy')
        e = np.load(f'{lowrankdir}/{sub}_task-emoidtop10_fc.npy')
        r = remap(vec2mat(r))
        n = remap(vec2mat(n))
        e = remap(vec2mat(e))
        race.append(ra)
        sex.append(se)
        age.append(ag)
        rest.append(r[a,b])
        nback.append(n[a,b])
        emoid.append(e[a,b])
        restmat.append(r)
        nbackmat.append(n)
        emoidmat.append(e)
    except:
        pass

rest = np.stack(rest)
nback = np.stack(nback)
emoid = np.stack(emoid)
restmat = np.stack(restmat)
nbackmat = np.stack(nbackmat)
emoidmat = np.stack(emoidmat)
race = np.array(race).astype('int')
sex = np.array(sex).astype('int')
age = np.array(age)

print([a.shape for a in [rest, nback, emoid, restmat, nbackmat, emoidmat, race, sex, age]])

[(1193, 34716), (1193, 34716), (1193, 34716), (1193, 264, 264), (1193, 264, 264), (1193, 264, 264), (1193,), (1193,), (1193,)]


In [23]:
# Load fake data

import pickle

fr = pickle.load(open('/home/anton/Documents/Tulane/Research/Work/Imputation/fake_rest200_5.pkl', 'rb'))
fr = np.stack(fr)

fn = pickle.load(open('/home/anton/Documents/Tulane/Research/Work/Imputation/fake_nback200_5.pkl', 'rb'))
fn = np.stack(fn)

fe = pickle.load(open('/home/anton/Documents/Tulane/Research/Work/Imputation/fake_emoid200_5.pkl', 'rb'))
fe = np.stack(fe)

fr2 = pickle.load(open('/home/anton/Documents/Tulane/Research/Work/Imputation/fake_rest600_5.pkl', 'rb'))
fr2 = np.stack(fr2)

fn2 = pickle.load(open('/home/anton/Documents/Tulane/Research/Work/Imputation/fake_nback264_5.pkl', 'rb'))
fn2 = np.stack(fn2)

fn3 = pickle.load(open('/home/anton/Documents/Tulane/Research/Work/Imputation/fake_nback264_500.pkl', 'rb'))
fn3 = np.stack(fn3)

fe2 = pickle.load(open('/home/anton/Documents/Tulane/Research/Work/Imputation/fake_emoid264_10.pkl', 'rb'))
fe2 = np.stack(fe2)

fakemats = np.concatenate([fr, fn, fe, fr2, fn2, fn3, fe2])

print(fakemats.shape)

(9069, 34716)


In [24]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.model_selection import train_test_split

class D(nn.Module):
    def __init__(self):
        super(D, self).__init__()
        self.fc1 = nn.Linear(34716, 20).float().cuda()
        self.fc2 = nn.Linear(20,1).float().cuda()

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x).squeeze()
        x = torch.stack([-x, x], dim=1)
        return x

realmats = np.concatenate([rest[:600], nback[:600], emoid[:600]])
x = np.concatenate([realmats, fakemats])
y = np.concatenate([np.zeros(len(realmats)), np.ones(len(fakemats))])

xtr, xt, ytr, yt = train_test_split(x, y, stratify=y, train_size=10000)

xtr = torch.from_numpy(xtr).float().cuda()
xt = torch.from_numpy(xt).float().cuda()

ytr = torch.from_numpy(ytr).long().cuda()

d = D()
optim = torch.optim.Adam(d.parameters(), lr=1e-4, weight_decay=1e-4)
ce = nn.CrossEntropyLoss()

nepochs = 2000
pperiod = 50

for e in range(nepochs):
    optim.zero_grad()
    yhat = d(xtr)
    loss = ce(yhat, ytr)
    loss.backward()
    optim.step()
    with torch.no_grad():
        d.fc1.weight[d.fc1.weight > 0.005] = 0.005
        d.fc1.weight[d.fc1.weight < -0.005] = -0.005
    if e % pperiod == 0 or e == nepochs-1:
        print(f'{e} {loss}')

print('Done')

with torch.no_grad():
    yhat = d(xt)[:,1]
    print(yhat)
    print(yt)

0 0.6278194785118103
50 0.23814228177070618
100 0.1870829313993454
150 0.16879408061504364
200 0.15908202528953552
250 0.15291330218315125
300 0.14850187301635742
350 0.14515312016010284
400 0.14230717718601227
450 0.1399369090795517
500 0.1378716081380844
550 0.13609644770622253
600 0.13438396155834198
650 0.13283832371234894
700 0.13157251477241516
750 0.13000349700450897
800 0.12882933020591736
850 0.12740984559059143
900 0.1263529360294342
950 0.12492922693490982
1000 0.12392811477184296
1050 0.1232224628329277
1100 0.1216764748096466
1150 0.12165412306785583
1200 0.11981888115406036
1250 0.1191234216094017
1300 0.11749304085969925
1350 0.11721867322921753
1400 0.11590132117271423
1450 0.11524125933647156
1500 0.11385588347911835
1550 0.11305997520685196
1600 0.1123821884393692
1650 0.11197952926158905
1700 0.1106303259730339
1750 0.10980301350355148
1800 0.1086861714720726
1850 0.1088864728808403
1900 0.10741230100393295
1950 0.10615807771682739
1999 0.10526706278324127
Done
tenso

In [25]:
torch.save(d.state_dict(), '/home/anton/Documents/Tulane/Research/Work/Imputation/disc7.torch')

print('Done')

Done
